# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [108]:
!type publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2020	Linking mineralogy and spectroscopy of highly aqueously altered CM and CI carbonaceous chondrites in preparation for primitive asteroid sample return	Meteoritics and Planetary Science	We collected visible and near-infrared (VNIR) and mid infrared (MIR) reflectance spectra from well-characterized CM1/2, CM1, and CI1 chondrites and identified trends related to their mineralogy and degree of secondary processing.	<b>Bates, H. C.,</b> King, A. J., Donaldson Hanna K. L., Bowles, N. E., and Russell, S. S.  (2020). Linking mineralogy and spectroscopy of highly aqueously altered CM and CI carbonaceous chondrites in preparation for primitive asteroid sample return, <i>Meteoritics and Planetary Science</i>. 55:77-101.	BatesCM1	http://hellybates.github.io/files/Bates2020.pdf
2021	A Spectral Investigation of Aqueously and Thermally Altered CM, CM-An, and CY Chondrites Under Simulated Asteroid Conditions for Comparison With OSIRIS-REx an

## Import pandas

We are using the very handy pandas library for dataframes.

In [109]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [143]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


pub_date                                              title  \
0  2020-01-01  Linking mineralogy and spectroscopy of highly ...   
1  2021-05-28  A Spectral Investigation of Aqueously and Ther...   
2  2019-12-01  The Yamato-type (CY) carbonaceous chondrite gr...   
3  2021-04-15  The aqueous alteration of CM chondrites, a review   
4  2022-11-16  The Winchcombe meteorite, a unique and pristin...   

                                      venue  excerpt  \
0         Meteoritics and Planetary Science      NaN   
1  Journal of Geophysical Research: Planets      NaN   
2                              Geochemistry      NaN   
3           Geochimica et Cosmochimica Acta      NaN   
4                          Science Advances      NaN   

                                            citation        url_slug  \
0  <b>Bates, H. C.,</b> King, A. J., Donaldson Ha...        BatesCM1   
1  <b>Bates, H. C.,</b> Donaldson Hanna K. L., Ki...       BatesSpec   
2  King, A. J., <b>Bates, H. C.,</b> Krietsch, D....         KingCYs   
3  Suttle, M. D., King, A. J., Schofield, P. F., ...        SuttleCM   
4  King., A. J., Daly, L., Rowe., R., Joy, K. H.,...  KingWinchcombe   

                                          paper_url  
0   http://hellybates.github.io/files/Bates2020.pdf  
1   http://hellybates.github.io/files/Bates2021.pdf  
2    http://hellybates.github.io/files/King2019.pdf  
3  http://hellybates.github.io/files/Suttle2021.pdf  
4    http://hellybates.github.io/files/King2022.pdf

## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [144]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [145]:
import os
for row, item in publications.iterrows():

    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    #md += "\nRecommended citation: " + item.citation
    md +=  item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [146]:
!dir C:\Users\heleb3\Documents\GitHub\hellybates.github.io\_publications\

 Volume in drive C has no label.
 Volume Serial Number is D0E7-FD37

 Directory of C:\Users\heleb3\Documents\GitHub\hellybates.github.io\_publications

15/06/2023  11:00    <DIR>          .
15/06/2023  11:00    <DIR>          ..
15/06/2023  11:18               856 2019-12-01-KingCYs.md
15/06/2023  11:18             1,056 2020-01-01-BatesCM1.md
15/06/2023  11:18               730 2021-04-15-SuttleCM.md
15/06/2023  11:18             1,169 2021-05-28-BatesSpec.md
15/06/2023  11:18               943 2022-11-16-KingWinchcombe.md
               5 File(s)          4,754 bytes
               2 Dir(s)  32,701,579,264 bytes free


In [147]:
!type ..\_publications\2020-01-01-BatesCM1.md

---
title: "Linking mineralogy and spectroscopy of highly aqueously altered CM and CI carbonaceous chondrites in preparation for primitive asteroid sample return"
collection: publications
permalink: /publication/2020-01-01-BatesCM1
date: 2020-01-01
venue: 'Meteoritics and Planetary Science'
paperurl: 'http://hellybates.github.io/files/Bates2020.pdf'
citation: '<b>Bates, H. C.,</b> King, A. J., Donaldson Hanna K. L., Bowles, N. E., and Russell, S. S.  (2020). Linking mineralogy and spectroscopy of highly aqueously altered CM and CI carbonaceous chondrites in preparation for primitive asteroid sample return, <i>Meteoritics and Planetary Science</i>. 55:77-101.'
---
[Download paper here](http://hellybates.github.io/files/Bates2020.pdf)
<b>Bates, H. C.,</b> King, A. J., Donaldson Hanna K. L., Bowles, N. E., and Russell, S. S.  (2020). Linking mineralogy and spectroscopy of highly aqueously altered CM and CI carbonaceous chondrites in preparation for primitive asteroid sample return, <i>Met